In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.signal import resample
import tensorflow as tf
from scipy.fftpack import fft,ifft
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
import os
import warnings
from tensorflow.keras import backend as K
import time
from tensorflow.keras.optimizers import Adam
from sklearn.decomposition import PCA
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from tensorflow.keras import models, optimizers, regularizers, initializers,constraints
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [ ]:
train = pd.read_csv('../dataset/sensor_train_final.csv')
test = pd.read_csv('../dataset/sensor_test_final.csv')
sub = pd.read_csv('../dataset/submit_example.csv')
y = train.groupby('fragment_id')['behavior_id'].min()

In [ ]:
train['mod'] = (train.acc_x ** 2 + train.acc_y ** 2 + train.acc_z ** 2) ** .5
train['modg'] = (train.acc_xg ** 2 + train.acc_yg ** 2 + train.acc_zg ** 2) ** .5
test['mod'] = (test.acc_x ** 2 + test.acc_y ** 2 + test.acc_z ** 2) ** .5
test['modg'] = (test.acc_xg ** 2 + test.acc_yg ** 2 + test.acc_zg ** 2) ** .5

train_fragment_len = len(train["fragment_id"].unique())
test_fragment_len = len(test["fragment_id"].unique())

x = np.zeros((train_fragment_len, 60, 8))
t = np.zeros((test_fragment_len, 60, 8))

for i in tqdm(range(15000)):
    tmp = train[train.fragment_id == i][:60]
    tmp = resample(tmp.drop(['fragment_id', 'time_point', 'behavior_id'],axis=1), 60, np.array(tmp.time_point))[0]
    x[i,:,:] = tmp
    
for i in tqdm(range(16000)):
    tmp = test[test.fragment_id == i][:60]
    tmp = resample(tmp.drop(['fragment_id', 'time_point'],axis=1), 60, np.array(tmp.time_point))[0]
    t[i,:,:] = tmp 

In [ ]:
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0

        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(shape=(input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        e = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))  # e = K.dot(x, self.W)
        if self.bias:
            e += self.b
        e = K.tanh(e)

        a = K.exp(e)
        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())
        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)

        c = K.sum(a * x, axis=1)
        return c

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.features_dim
    
    def get_config(self):
        config = {
            "init":self.init,
            "supports_masking":self.supports_masking,
            "W_regularizer":self.W_regularizer,
            "b_regularizer":self.b_regularizer,
            "W_constraint":self.W_constraint,
            "b_constraint":self.b_constraint,
            "bias":self.bias,
            "step_dim":self.step_dim,
            "features_dim":self.features_dim,
        }
        base_config = super(Attention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
def Net():
    input1 = Input(shape=(60, 8),name="rnn")

    x1 = Bidirectional(GRU(128,activation="tanh", return_sequences=True), merge_mode='concat')(input1)
    x1 = Dropout(0.5)(x1)
    x1 = BatchNormalization()(x1)

    x2 = Bidirectional(GRU(128,activation="tanh", return_sequences=True), merge_mode='concat')(x1)
    x2 = Dropout(0.5)(x2)
    x2 = BatchNormalization()(x2)

    x3 = Bidirectional(GRU(128,activation="tanh", return_sequences=True), merge_mode='concat')(x2)
    x3 = Dropout(0.5)(x3)
    x3 = BatchNormalization()(x3)
    X = concatenate([x1,x2,x3])
    
    X = Attention(60)(X)
    X_L = X.shape[1]
    X = Dense(X_L*4, activation="relu")(X)
    X = Dense(X_L*1, activation="relu")(X)
    X = Dropout(0.5)(X)
    X = Dense(20, activation="softmax")(X)

    return Model([input1], X)

In [ ]:
k=20
kfold = StratifiedKFold(k, shuffle=True,  random_state=256)
proba_t = np.zeros((16000, 20))
for fold, (xx, yy) in enumerate(kfold.split(x, y)):
    y_ = to_categorical(y, num_classes=20)
    model = Net()
    model.compile(
                optimizer=Adam(),
                loss="categorical_crossentropy",
                metrics=['accuracy']
                )
    callbacks = [
        ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.5, patience = 8,verbose = 0, mode = 'max'),
        EarlyStopping(monitor = 'val_accuracy', patience = 18, mode = 'max', verbose = 0),
                ]
    strategy = tf.distribute.MirroredStrategy()
    
    model.fit({
                "rnn":x[xx]
                },
              y_[xx],
              epochs=150,
              batch_size=128,
              verbose=1,
              shuffle=True,
              validation_data=({
                "rnn":x[yy]
              }, y_[yy]),
              callbacks=callbacks,
             )
    
    proba_t += model.predict({
                "rnn":t
                }, verbose=0, batch_size=128) / k
    

In [ ]:
np.save('gru_pred.npy', proba_t)